In [15]:
import bs4
from dotenv import load_dotenv
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 환경변수 가져오기
load_dotenv()

# 웹문서 데이터 로더 선언(WebBaseLoader)
loader = WebBaseLoader(
    web_path=("https://n.news.naver.com/article/437/0000416134"),
    bs_kwargs=dict(
        # 특정 요소에서만 파싱하도록 제한하는 필터 적용
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class": ["newsct_article _article_body", "media_end_head_go_trans"]}
        )
    )
)

docs = loader.load()

# print(docs)

# 문서 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=10)
splits = text_splitter.split_documents(docs)

# 임베딩 생성
embeddings = OpenAIEmbeddings()

# 벡터스토어 생성
vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)

# 검색기(retriever) 생성
retriever = vectorstore.as_retriever()

# runtime

# 프롬프트
prompt = PromptTemplate.from_template("""
    질문-답변을 수행하는 AI 어시스턴트를 수행한다.
    주어진 문맥에 검색된 다음 문맥{context}를 사용하여 질문에 답한다.
    만약 주어진 문맥에서 답을 찾을 수 없는 경우 모른다고 답변한다.
    한글로 답변한다.
    
    #Question:
    {question}
    
    #Context
    {context}
    
    #Answer:
""")

# LLM 모델
llm = ChatOpenAI(model="gpt-4o", temperature=1.0)

# chain 구성
news_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 질문하기(사용하기)
answer = news_chain.invoke("로제는 아파트 저작권료로 서울 아파트를 살 수 있을까?")

print(answer)

모른다.
